# Create Telemetry Dataset 

## Lets track wolves

In [6]:
import pandas as pd
import geopandas as gpd
import names
import datetime
import random
from shapely.geometry import shape, Polygon, Point

### Tables

#### Packs
    - Pack ID (Int)
    - Pack Name (Text) 
    - Pack Extent (Geometry - Polygon)

#### Wolves
    - Wolf ID (Int)
    - Sex (Text)
    - Born (Date)
    - Died (Data)

#### PackMembers
    - Pack ID 
    - Wolf ID

#### TrackedWold 
    - Wolf ID (Int)
    - Collar ID (Int)
    - DateAdded (DateTime)
    - Date Removed

#### Captured 
    - Wolf Id (Int)
    - Weight (Float)
    - Health (Int - Domain)
    - CaptureLocation (Geometry - Point)

#### HealthCondition
    - HealthCode (Int)
    - HealthDescription (Text)
    - Comments (Text)

#### Observations
    - CollarID (Int)
    - DateTime (DateTime)
    - Location (Geometry - Point)

In [7]:
end_date = datetime.date.today()
avg_life_span_days = 14 * 365
start_date = end_date - datetime.timedelta(avg_life_span_days)

countPacks = 4
countWolves = 37
countCollars = 12

male_avg_weight = 79
female_weight_difference_range = (5,10)


In [8]:
class Pack: 
    def __init__(self, PackID, PackName, PackExtent,) -> None:
        self.PackID = PackID
        self.PackName = PackName
        PackExtent = PackExtent
        
    
class Wolves:
    def __init__(self,  WolfID) -> None:
        self.WolfID = WolfID
        self.Sex = self.get_Sex()
        self.WolfName = self.get_name()
        self.Born = self.get_birth()
        self.Died = self.get_Death()
    
    def get_Sex(self):
        
        if random.randrange(0, 10) >= 5:
            sex = 'male'
        else: 
            sex = 'female'
            
        return sex
        

    def get_name(self):
        return names.get_first_name(gender=self.Sex)
    
    def get_birth(self):
        random_number_of_days = random.randrange(avg_life_span_days)
        random_date = start_date + datetime.timedelta(days=random_number_of_days)
        return random_date
    
    def get_Death(self):
        if random.randrange(10) > 5:
            random_number_of_days = random.randrange(avg_life_span_days)
            if (self.Born + datetime.timedelta(days=random_number_of_days)) < end_date:
                death = self.Born + datetime.timedelta(days=random_number_of_days)
            else: 
                death = None
            return death
        

class PackMembers(Wolves):
    def __init__(self, wolfID) -> None:
        super().__init__(wolfID)
        self.PackID = self.get_PackID()
        
    def get_PackID(self):
        return random.randrange(countPacks)
    

        
class TrackedWolf(PackMembers):
    def __init__(self, wolfID) -> None:
        super().__init__(wolfID)
        self.CollarID = self.get_Collar()
        self.DateAdded = self.get_DateAdded()
        self.DateRemoved = self.get_DateRemoved()

    def get_Collar(self):
        return random.randrange(countCollars)
    
    def get_DateAdded(self):
        
        random_number_of_days = random.randrange(avg_life_span_days)
        random_date = start_date + datetime.timedelta(days=random_number_of_days)
        
        return random_date
    
    def get_DateRemoved(self):
        
        if random.randrange(0,10) > 3:
            time_between_dates = end_date - self.DateAdded
            random_number_of_days = random.randrange(time_between_dates.days)
            random_date = self.DateAdded + datetime.timedelta(days=random_number_of_days)
        else: 
            random_date = None
        
        return random_date

class HealthCondition:
    def __init__(self,HealthCode, HealthDescription, Comments) -> None:
        self.HealthCode = HealthCode
        self.HealthDescription = HealthDescription  
        self.Comments = Comments 

class Captures(TrackedWolf):
    def __init__(self, CollardID, CaptureDate, HealthCondition, PackExtent) -> None:
        super().__init__(CollardID)
        self.Date = CaptureDate if CaptureDate is not None else self.DateAdded
        self.Weight = self.get_Weight()
        self.Health = HealthCondition
        self.PackExtent = PackExtent
        self.CaptureLocation = self.get_CaptureLocation()
            
    def get_Weight(self):
        if self.Sex == 'Male':
            weight = male_avg_weight + random.uniform(-3,5)
        else:
            weight = random.uniform(male_avg_weight - female_weight_difference_range[0], male_avg_weight - female_weight_difference_range[1])
        
        return weight
    
    def get_CaptureLocation(self):
        minx, miny, maxx, maxy = self.PackExtent
        return Point(random.uniform(minx, maxx), random.uniform(miny, maxy))



class Observations(TrackedWolf):
    def __init__(self, CollardID, PackExtent, end_date) -> None:
        super().__init__(CollardID)
        self.end_date = end_date
        self.PackExtent = PackExtent
        self.Pings = []
        self.get_Pings()

    def get_PingInterval(self):
        random_number_of_days = random.randrange(0,2)
        random_number_of_hours = random.randrange(1,4) * 4
            
        return datetime.timedelta(days=random_number_of_days, hours=random_number_of_hours)
    
    def get_GPS(self):
        
        minx, miny, maxx, maxy = self.PackExtent
        
        return Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
            
    def get_Pings(self):
        end_date = self.DateRemoved if self.DateRemoved is not None else self.end_date 
        tracking_date_range = self.end_date - self.DateAdded
        
        PingDate = start_date + self.get_PingInterval()
        
        while PingDate < end_date:
            self.Pings.append({"Date" : PingDate, "Location" : self.get_GPS()})
            PingDate = PingDate + self.get_PingInterval()
              
                



In [9]:
wolfpacks = "/Users/ryan/Development/Intro_To_GIS-ENVR_250/CourseMaterials/Labs/Week_4/pack_locations.geojson"
wolfpacks_gdf = gpd.read_file(wolfpacks).set_index("PackID")
wolfpacks_gdf.to_csv("wolfpacks.csv")

In [10]:
wolves = []
i = 0 
while i < countWolves:
    wolves.append(Wolves(i).__dict__)
    i = i + 1

In [11]:
wolves_df = pd.DataFrame(wolves).set_index("WolfID")
wolves_df.head()
wolves_df.to_csv("wolves.csv")

In [12]:
pack_members = [PackMembers(wolf.get("WolfID")).__dict__ for wolf in wolves]

In [13]:
pack_members_df = pd.DataFrame(pack_members).drop(columns=["Sex", "WolfName", "Born", "Died"]).set_index("WolfID")
pack_members_df.head()
pack_members_df.to_csv("pack_members.csv")

In [14]:
tracked_wolves = [TrackedWolf(random.randrange(countWolves)).__dict__ for i in range(1,20)]

In [15]:
tracked_wolves_df = pd.DataFrame(tracked_wolves).drop(columns=["Sex", "WolfName", "Born", "Died", "PackID"]).set_index("WolfID")
tracked_wolves_df.head()
tracked_wolves_df.to_csv("tracked_wolves.csv")

In [16]:
code = [0, 1, 2, 3]
description = ['poor', 'fair', 'good', 'excellent']
comments = ["real bad shape", "could use more prey", "goood access to prey", "this ones fat"]

conditions = []

for k,v,d in zip(code, description, comments):
    conditions.append(HealthCondition(k,v,d).__dict__)
    

In [17]:
conditions_df = pd.DataFrame(conditions)
conditions_df.head()
conditions_df.to_csv("conditions.csv")

In [18]:
# wolfpacks_gdf.apply(lambda x: print(x), axis= 1)
captured_wolves = []


# add data for when collar was added 
for wolf in tracked_wolves:
    this_wolfs_pack = wolfpacks_gdf[wolfpacks_gdf.index == wolf.get("PackID")].bounds
    captured_wolves.append(Captures(wolf.get("WolfID"), None, conditions[random.randrange(4)].get("HealthCode"), this_wolfs_pack.values.tolist()[0]).__dict__)
    


In [19]:
# add some random data too.

for i in range(0,int(avg_life_span_days/365), 2):
    for w in range(0,4):
        wolf = random.randrange(countWolves)
        this_wolf = wolves_df[ wolves_df.index == wolf]
        this_wolf_pack = pack_members_df[pack_members_df.index== wolf]
        this_wolfs_pack_extent = wolfpacks_gdf[wolfpacks_gdf.index == this_wolf_pack.PackID.values[0]].bounds
        
        if this_wolf.Died.values[0] is not None:
            
            random_number_of_days = random.randrange(365*2)
            random_date = this_wolf.Born.values[0] + datetime.timedelta(days=random_number_of_days)

            captured_wolves.append(Captures(wolf, random_date, conditions[random.randrange(4)].get("HealthCode"), this_wolfs_pack_extent.values.tolist()[0]).__dict__)
        
        
    

In [26]:
captured_wolves_df = pd.DataFrame(captured_wolves).set_index("WolfID")
captured_wolves_df.drop(columns=["Sex", "WolfName", "Born", "Died", "PackID", "CollarID",  "DateAdded", "DateRemoved", "PackExtent"], inplace=True)
captured_wolves_df.to_csv("captured_wolves.csv")
captured_wolves_df.head()

,Date,Weight,Health,CaptureLocation
WolfID,,,,
22,2012-03-24,69.970798,3,POINT (-117.59531925198397 48.50211457494423)
23,2015-03-23,69.197925,3,POINT (-117.30386815076224 46.06596756797705)
23,2011-01-14,72.155479,3,POINT (-117.33624040437321 46.0305371714037)
12,2012-08-29,70.389906,1,POINT (-117.32488405310612 46.10024472441666)
12,2014-07-03,70.184938,0,POINT (-120.96417179722003 47.32279261457643)


In [21]:
obs = []

for wolf in tracked_wolves:
    this_wolfs_pack = wolfpacks_gdf[wolfpacks_gdf.index == wolf["PackID"]].bounds
    obs.append(Observations(wolf.get("CollarID"), this_wolfs_pack.values.tolist()[0], end_date).__dict__)
obs_df = pd.DataFrame(obs)
obs_df.head()

,WolfID,Sex,WolfName,Born,Died,PackID,CollarID,DateAdded,DateRemoved,end_date,PackExtent,Pings
0,0,female,Joan,2019-10-25,None,1,3,2014-08-08,2019-11-08,2022-02-06,"[-117.8338623046875, 48.16974908365419, -117.3...","[{'Date': 2008-02-11, 'Location': POINT (-117...."
1,0,female,Lydia,2019-07-28,None,2,11,2009-01-02,2011-07-20,2022-02-06,"[-117.45208740234374, 46.00840867976967, -117....","[{'Date': 2008-02-10, 'Location': POINT (-117...."
2,4,male,Allen,2016-05-10,None,3,8,2008-07-19,2018-02-20,2022-02-06,"[-117.45208740234374, 46.00840867976967, -117....","[{'Date': 2008-02-11, 'Location': POINT (-117...."
3,5,female,Renee,2020-03-09,None,3,9,2009-07-09,None,2022-02-06,"[-117.45208740234374, 46.00840867976967, -117....","[{'Date': 2008-02-10, 'Location': POINT (-117...."
4,6,male,Otis,2009-10-04,2013-03-20,1,1,2020-01-24,2020-03-28,2022-02-06,"[-120.97869873046875, 47.22143353240336, -120....","[{'Date': 2008-02-11, 'Location': POINT (-120...."


In [22]:

observations = obs_df[["WolfID", "Pings"]].apply(pd.Series.explode).reset_index(drop=True).set_index("WolfID")


observations["Date"] =  observations[["Pings"]].apply(lambda x: dict(x).get("Pings").get("Date"), axis=1)
observations["Location"] =  observations[["Pings"]].apply(lambda x: dict(x).get("Pings").get("Location"), axis=1)
observations_df = observations.drop(columns=["Pings"])
observations_df.to_csv("wolf_observations.csv")

/opt/anaconda3/envs/base_gis/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [29]:
data = wolves_df.join(pack_members_df).join(wolfpacks_gdf, on="PackID").join(tracked_wolves_df, on="WolfID").join(captured_wolves_df).join()

data

,Sex,WolfName,Born,Died,PackID,PackName,geometry,CollarID,DateAdded,DateRemoved,Date,Weight,Health,CaptureLocation
WolfID,,,,,,,,,,,,,,
0,female,Marion,2021-06-07,None,3,Blue,"POLYGON ((-117.31476 46.03511, -117.30103 46.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,male,Bryan,2019-09-26,None,2,Cle Elum,"POLYGON ((-120.86884 47.22143, -120.78918 47.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,male,Frank,2019-01-15,None,0,Colville,"POLYGON ((-117.49329 48.59296, -117.70203 48.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,male,Philip,2014-08-21,None,2,Cle Elum,"POLYGON ((-120.86884 47.22143, -120.78918 47.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,male,Robert,2012-04-18,2016-02-18,1,Okanogan,"POLYGON ((-119.88556 48.38362, -119.90479 48.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,male,Todd,2017-08-03,None,0,Colville,"POLYGON ((-117.49329 48.59296, -117.70203 48.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,male,George,2019-10-09,None,2,Cle Elum,"POLYGON ((-120.86884 47.22143, -120.78918 47.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,female,Katherine,2018-11-05,None,3,Blue,"POLYGON ((-117.31476 46.03511, -117.30103 46.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
data.to_csv("test.csv")